Checkpoint Objective

Implement KNN from scratch 

The test problem we will be using in this tutorial is the iris classification.

The problem consists of 150 observations of iris flowers from three different species. There are 4 measurements of given flowers: sepal length, sepal width, petal length, and petal width (all in the same unit of centimeters). The predicted attribute is the species, which is either Setosa, Versicolor, or Virginica.

In [15]:
import csv
import math
import numpy as np
import operator #operator module for sorting
import random
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

1. Handle Data

In [2]:
#loading the data file
with open('iris.data.txt', 'r') as csvfile:
        lines = csv.reader(csvfile)
        for row in lines:
            print(', '.join(row))

5.1, 3.5, 1.4, 0.2, Iris-setosa
4.9, 3.0, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.3, 0.2, Iris-setosa
4.6, 3.1, 1.5, 0.2, Iris-setosa
5.0, 3.6, 1.4, 0.2, Iris-setosa
5.4, 3.9, 1.7, 0.4, Iris-setosa
4.6, 3.4, 1.4, 0.3, Iris-setosa
5.0, 3.4, 1.5, 0.2, Iris-setosa
4.4, 2.9, 1.4, 0.2, Iris-setosa
4.9, 3.1, 1.5, 0.1, Iris-setosa
5.4, 3.7, 1.5, 0.2, Iris-setosa
4.8, 3.4, 1.6, 0.2, Iris-setosa
4.8, 3.0, 1.4, 0.1, Iris-setosa
4.3, 3.0, 1.1, 0.1, Iris-setosa
5.8, 4.0, 1.2, 0.2, Iris-setosa
5.7, 4.4, 1.5, 0.4, Iris-setosa
5.4, 3.9, 1.3, 0.4, Iris-setosa
5.1, 3.5, 1.4, 0.3, Iris-setosa
5.7, 3.8, 1.7, 0.3, Iris-setosa
5.1, 3.8, 1.5, 0.3, Iris-setosa
5.4, 3.4, 1.7, 0.2, Iris-setosa
5.1, 3.7, 1.5, 0.4, Iris-setosa
4.6, 3.6, 1.0, 0.2, Iris-setosa
5.1, 3.3, 1.7, 0.5, Iris-setosa
4.8, 3.4, 1.9, 0.2, Iris-setosa
5.0, 3.0, 1.6, 0.2, Iris-setosa
5.0, 3.4, 1.6, 0.4, Iris-setosa
5.2, 3.5, 1.5, 0.2, Iris-setosa
5.2, 3.4, 1.4, 0.2, Iris-setosa
4.7, 3.2, 1.6, 0.2, Iris-setosa
4.8, 3.1, 1.6, 0.2, Iris-setosa
5.4, 3.4

In [5]:
#spliting the data into a training dataset 


def loadDataset(filename, split, trainingSet=[], testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset) - 1):
            # Convert the first 4 columns to floats
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            
            # Randomly split the data based on the 'split' parameter
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])
                
#testing this function
trainingSet=[]
testSet=[]
loadDataset('iris.data.txt', 0.66, trainingSet, testSet)
print ('Train: ' + repr(len(trainingSet)))
print ('Test: ' + repr(len(testSet)) )

Train: 98
Test: 51


2. Similarity

To make predictions we need to calculate the similarity between any two given data instances. This is needed so that we can locate the k most similar data instances in the training dataset for a given member of the test dataset and in turn, make a prediction.

Given that all four flower measurements are numeric and have the same units, we can directly use the Euclidean distance measure. 

Additionally, we want to control which fields to include in the distance calculation. Specifically, we only want to include the first 4 attributes. One approach is to limit the Euclidean distance to a fixed length, ignoring the final dimension.

Putting all of this together, you have to define the Euclidean distance

In [6]:

def euclideanDistance(instance1, instance2, length):
    arr1 = np.array(instance1[:length])
    arr2 = np.array(instance2[:length])
    
    if len(arr1) != len(arr2):
        raise ValueError()
    
    return np.linalg.norm(arr1 - arr2)

# Example :
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']
distance = euclideanDistance(data1, data2, 3)
print('Distance:', repr(distance))


Distance: 3.4641016151377544


3. Neighbors

Now that we have a similarity measure, we can use it to collect the k most similar instances for a given unseen instance.

This is a straightforward process of calculating the distance for all instances and selecting a subset with the smallest distance values.

Below is the getNeighbors function that returns k most similar neighbors from the training set for a given test instance (using the already defined euclideanDistance function)

In [7]:
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance) - 1
    
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))

    distances.sort(key=operator.itemgetter(1))
    
    neighbors = []
    
    for x in range(k):
        neighbors.append(distances[x][0])
    
    return neighbors

# Example:
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
k = 1

neighbors = getNeighbors(trainSet, testInstance, k)
print(neighbors)


[[4, 4, 4, 'b']]


4. Response

Once we have located the most similar neighbors for a test instance, the next task is to devise a predicted response based on those neighbors.

We can do this by allowing each neighbor to vote for their class attribute, and taking the majority vote as the prediction.

Below is a function for getting the majority voted response from a number of neighbors. It assumes the class is the last attribute for each neighbor.

In [8]:
def getResponse(neighbors):
    classVotes = {}
    
    for x in range(len(neighbors)):
        response = neighbors[x][-1]  # Get the class label from the last element
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

# Example :
neighbors = [[1, 1, 1, 'a'], [2, 2, 2, 'a'], [3, 3, 3, 'b']]
response = getResponse(neighbors)
print("Response is:",response)

Response is: a


5. Accuracy

We have all of the pieces of the KNN algorithm in place. An important remaining concern is how to evaluate the accuracy of predictions.

An easy way to evaluate the accuracy of the model is to calculate a ratio of the total correct predictions out of all predictions made, called classification accuracy.

Below is the getAccuracy function that sums the total correct predictions and returns the accuracy as a percentage of correct classifications.

In [9]:
def getAccuracy(testSet, predictions):
    correct = 0
    
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1

    accuracy = (correct / float(len(testSet))) * 100.0
    return accuracy

# Example:
testSet = [[1, 1, 1, 'a'], [2, 2, 2, 'a'], [3, 3, 3, 'b']]
predictions = ['a', 'a', 'a']
accuracy = getAccuracy(testSet, predictions)
print("Accuracy: ",accuracy)



Accuracy:  66.66666666666666


6. Main

In [13]:
# Function to load the Iris dataset and split it into training and testing sets
def loadDataset(filename, split):
    trainingSet = []
    testSet = []
    
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        
        for x in range(len(dataset) - 1):
            # Convert the first 4 columns to floats
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
            
            # Randomly split the data based on the 'split' parameter
            if random.random() < split:
                trainingSet.append(dataset[x])
            else:
                testSet.append(dataset[x])
    
    return trainingSet, testSet

# Main function for the Iris dataset
def main():
    
    filename = 'iris.data.txt'
    split = 0.67 
    trainingSet, testSet = loadDataset(filename, split)

    k = 3  
    knn = KNeighborsClassifier(n_neighbors=k)

   
    X_train = [row[:-1] for row in trainingSet]
    y_train = [row[-1] for row in trainingSet]

    knn.fit(X_train, y_train)

    X_test = [row[:-1] for row in testSet]

    predictions = knn.predict(X_test)

    correct = sum(1 for true, pred in zip([row[-1] for row in testSet], predictions) if true == pred)
    accuracy = (correct / len(testSet)) * 100.0
    print(f"Accuracy: {accuracy:.2f}%")

if __name__ == "__main__":
    main()


Accuracy: 94.23%


C:\Users\beit\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


7. Another distance metric

In [16]:
#i'm going to use the manhattan distance
def main():
    filename = 'iris.data.txt'
    split = 0.67
    k = 3

    trainingSet, testSet = loadDataset(filename, split)
    
    # using the  Manhattan distance
    def manhattan_distance(x, y):
        return sum(abs(a - b) for a, b in zip(x, y))
    
    knn = KNeighborsClassifier(n_neighbors=k, metric=manhattan_distance)
    
    X_train, y_train = zip(*[(row[:-1], row[-1]) for row in trainingSet])
    X_test, y_test = zip(*[(row[:-1], row[-1]) for row in testSet])

    knn.fit(X_train, y_train)
    predictions = knn.predict(X_test)
    
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy: {accuracy * 100:.2f}%")

if __name__ == "__main__":
    main()

Accuracy: 97.96%


C:\Users\beit\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
